In [1]:
import pandas as pd
import numpy as np

In [2]:
imports = pd.read_csv('../cp_complete.csv')

In [3]:
ruta_medicamentos = '../medicamentosHomologado.csv'
medicines = pd.read_csv(ruta_medicamentos)

In [4]:
#creacion de columnas para ir detectando cambios y más
imports['cnis'] = ''
imports['status_medicamento'] = ''
imports['empresa'] = ''
imports['unidosis'] = ''
imports['c_r_c_llenado_datos'] = ''

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE y MARCA
medicines[['CONCENTRACION','UNIDAD CONCENTRACION','PRESENTACION','FORMA FARMA']][ (medicines['MEDICAMENTO'] == 'ruxolitinib' ) & (medicines['NOMBRE COMERCIAL']=='jakavi') ]

In [ ]:
#retorna info relevante de medicamentos segun su NOMBRE 
medicines[['CONCENTRACION','UNIDAD CONCENTRACION','FORMA FARMA','MEDICAMENTO','NOMBRE COMERCIAL']][ medicines['MEDICAMENTO'] == 'everolimus' ]

In [4]:
#es necesario porque concentracion era float64 y le agregaba .0 a los numeros enteros y un 0.5 lo hacia 0
imports['concentracion'] =  imports['concentracion'].astype(str).str.replace(".0","",regex=False)

In [5]:
medicines['CONCENTRACION'] =  medicines['CONCENTRACION'].astype(str).str.replace(".0","",regex=False)

## funciones

In [6]:
nuevos_medicamentos = {}

In [7]:
def match_imports_medicine(imp_row: pd.Series) -> list:
    """ 
    receive a row of imports, will try to do a match in base of 
    nombre,marca and presentation of medicine in cuestion between 
    imports and medicines dataframes."""
    #checamos si hace match perfecto(nombre,marca,presentacion etc)
    primer_intento =  medicines[(medicines['MEDICAMENTO']==imp_row['nombre']) & 
                                (medicines['NOMBRE COMERCIAL']==imp_row['marca']) &
                                (medicines['CONCENTRACION']==imp_row['concentracion']) & 
                                (medicines['UNIDAD CONCENTRACION']==imp_row['unidad_concentracion']) & 
                                (medicines['PRESENTACION']==imp_row['presentacion']) & 
                                (medicines['FORMA FARMA']==imp_row['forma_farma']) ].to_dict('records')
    if len(primer_intento) > 0:
        #print(f"match perfecto {len(primer_intento)}")
        return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],imp_row['nombre'],
                imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],
                primer_intento[0]['EMPRESA'], primer_intento[0]['CONCENTRACION'], primer_intento[0]['UNIDAD CONCENTRACION'],
                primer_intento[0]['PRESENTACION'], primer_intento[0]['FORMA FARMA'], primer_intento[0]['CNIS'], primer_intento[0]['STATUS'],'perfect match',None)
    
    #si no hizo match perfecto hay que ver que le falta y cuantos match con el mayor numero de coincidencias hace
    df_dict = medicines[ (medicines['MEDICAMENTO']==imp_row['nombre']) & (medicines['NOMBRE COMERCIAL']==imp_row['marca']) ].to_dict('records')
    if len(df_dict) == 0:
        print(f"sin match para{imp_row['nombre']}, {imp_row['marca']}")
        return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],imp_row['nombre'],
                imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,'sin match',None)
    
    matches = 0 # número de matches entre imports y medicamentos con el mismo numero de coincidencias
    coincidencias = 0 #coincidencia entre imports y medicamentos en sus columnas, concentracion,unidad_concentracion, presentacion, forma farma
    medicine_match = '' #concatenacion del match o matches según sea el caso
    gb_dismatch = '' #es una concatenacion de pares claves que indican donde se fdió la diferencia en el match o matches
    match_ind = [] 
    global nuevos_medicamentos #genera un arreglo con medicamento y su número de apariciones
    for i,medic in enumerate(df_dict):
        contador = 0
        dismatch = ''
        if medic['CONCENTRACION'] == imp_row['concentracion']:
            contador += 1
        else:
            dismatch = 'cc' 
        if medic['UNIDAD CONCENTRACION'] == imp_row['unidad_concentracion']:
            contador += 1
        else:
            dismatch += 'uc' if dismatch == '' else '|uc'
        if medic['PRESENTACION'] == imp_row['presentacion']:
            contador += 1
        else:
            dismatch += 'pp' if dismatch == '' else '|pp'
        if medic['FORMA FARMA'] == imp_row['forma_farma']:
            contador += 1
        else:
            dismatch += 'ff' if dismatch == '' else '|ff'

        #si hay coincidencias y es mayor en numero de estas que el match anterior(en caso de que halla)
        if contador > coincidencias:
            coincidencias = contador
            matches = 1     
            medicine_match = str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch = dismatch
            match_ind = [i]
        #si existen las mismas coincidencias que el match anterior 
        elif contador > 0 and contador == coincidencias:
            matches += 1    
            medicine_match += " || " + str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch += " || " + dismatch
            match_ind.append(i)

    c_r_c = f"{coincidencias} coincidencias en {matches} matches: {medicine_match}"

    if matches == 1 and coincidencias == 3:
        ind = match_ind[0]
        if gb_dismatch == 'pp' and imp_row['presentacion'] == 'sp':
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
        elif gb_dismatch == 'ff':
            msj = f"se forzó su forma farma de {imp_row['forma_farma']} a {df_dict[ind]['FORMA FARMA']} dado el caso {c_r_c}"
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],
                    df_dict[ind]['EMPRESA'], df_dict[ind]['CONCENTRACION'], df_dict[ind]['UNIDAD CONCENTRACION'],
                    df_dict[ind]['PRESENTACION'], df_dict[ind]['FORMA FARMA'], df_dict[ind]['CNIS'], df_dict[ind]['STATUS'],'forzed perfect match',msj)
        elif gb_dismatch == 'pp' and imp_row['presentacion'] != 'sp':
            new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(df_dict[ind]['CONCENTRACION']) +' '+ 
                            df_dict[ind]['UNIDAD CONCENTRACION'] +' '+ str(imp_row['presentacion']) +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
            if new_medicine not in nuevos_medicamentos:     
                nuevos_medicamentos[new_medicine] = 1
            else:
                nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine] + 1
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
        elif gb_dismatch == 'cc':
            new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(imp_row['concentracion']) +' '+ 
                            df_dict[ind]['UNIDAD CONCENTRACION'] +' '+ str(df_dict[ind]['PRESENTACION'])  +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
            if new_medicine not in nuevos_medicamentos:
                nuevos_medicamentos[new_medicine] = 1
            else:
                nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine]+1
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)

    elif (matches == 3 and coincidencias == 3) or (matches == 2 and coincidencias == 3):
        if (gb_dismatch == 'cc || cc || cc') or (gb_dismatch == 'cc || cc'):
            ind = match_ind[0]
            new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(imp_row['concentracion']) +' '+ 
                            df_dict[ind]['UNIDAD CONCENTRACION'] +' '+ str(df_dict[ind]['PRESENTACION'])  +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
            if new_medicine not in nuevos_medicamentos:
                nuevos_medicamentos[new_medicine] = 1
            else:
                nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine]+1
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
        else:
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
    
    elif matches == 2 and coincidencias == 2:
        if imp_row['presentacion'] == 'sp':
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
        elif imp_row['presentacion'] != 'sp':
            if gb_dismatch == 'pp|ff || pp|ff':
                ind = match_ind[0]
                new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(df_dict[ind]['CONCENTRACION']) +' '+ 
                            df_dict[ind]['UNIDAD CONCENTRACION'] +' '+ str(imp_row['presentacion']) +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
                if new_medicine not in nuevos_medicamentos:
                    nuevos_medicamentos[new_medicine] = 1
                else:
                    nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine]+1
                return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                        imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                        imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)
        else:
            print("somethign went wrong con 2 mat y 2 coinc")

    else:
        #gb_dismatch = medic['FORMA FARMA'] + " || " + imp_row['forma_farma'] if gb_dismatch == 'ff' else gb_dismatch
        return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)

## match

In [8]:
avertl = imports[imports['nombre_y_marca']=='NM'].apply(match_imports_medicine, axis=1, result_type='expand').copy()

In [16]:
avertl.to_csv('prueba.csv')

In [ ]:
avertl[(avertl[17] == 'perfect match') | (avertl[17] == 'forzed perfect match')]

In [17]:
avertl[(avertl[17] != 'perfect match') & (avertl[17] != 'forzed perfect match')].to_csv('conflictos.csv')

In [9]:
nuevos_medicamentos

{'larotrectinib vitrakvi Bayer 100 mg 100 capsulas nan': 1}

#### pruebita

In [22]:
nuevos_medicamentos = {}

In [23]:

def match_imports_medicine_2(imp_row: pd.Series) -> list:
    """ 
    receive a row of imports, will try to do a match in base of 
    nombre,marca and presentation of medicine in cuestion between 
    imports and medicines dataframes."""
    #checamos si hace match perfecto(nombre,marca,presentacion etc)
    primer_intento =  medicines[(medicines['MEDICAMENTO']==imp_row['nombre']) & 
                                (medicines['NOMBRE COMERCIAL']==imp_row['marca']) &
                                (medicines['CONCENTRACION']==imp_row['concentracion']) & 
                                (medicines['UNIDAD CONCENTRACION']==imp_row['unidad_concentracion']) & 
                                (medicines['PRESENTACION']==imp_row['presentacion']) & 
                                (medicines['FORMA FARMA']==imp_row['forma_farma']) ].to_dict('records')
    if len(primer_intento) > 0:
        #print(f"match perfecto {len(primer_intento)}")
        return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],imp_row['nombre'],
                imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],
                primer_intento[0]['EMPRESA'], primer_intento[0]['CONCENTRACION'], primer_intento[0]['UNIDAD CONCENTRACION'],
                primer_intento[0]['PRESENTACION'], primer_intento[0]['FORMA FARMA'], primer_intento[0]['CNIS'], primer_intento[0]['STATUS'],'perfect match',None)
    
    #si no hizo match perfecto hay que ver que le falta y cuantos match con el mayor numero de coincidencias hace
    df_dict = medicines[ (medicines['MEDICAMENTO']==imp_row['nombre']) & (medicines['NOMBRE COMERCIAL']==imp_row['marca']) ].to_dict('records')
    if len(df_dict) == 0:
        print(f"sin match para{imp_row['nombre']}, {imp_row['marca']}")
        return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],imp_row['nombre'],
                imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,'sin match',None)
    
    matches = 0 # número de matches entre imports y medicamentos con el mismo numero de coincidencias
    coincidencias = 0 #coincidencia entre imports y medicamentos en sus columnas, concentracion,unidad_concentracion, presentacion, forma farma
    medicine_match = '' #concatenacion del match o matches según sea el caso
    gb_dismatch = '' #es una concatenacion de pares claves que indican donde se fdió la diferencia en el match o matches
    match_ind = [] #arreglo de indices de matches en df_dict
    global nuevos_medicamentos #genera un arreglo con medicamento y su número de apariciones
    for i,medic in enumerate(df_dict):
        contador = 0
        dismatch = ''
        if medic['CONCENTRACION'] == imp_row['concentracion']:
            contador += 1
        else:
            if imp_row['concentracion'] == 'sc':
                dismatch = 'sc' 
            else:
                dismatch = 'cc' 
        if medic['UNIDAD CONCENTRACION'] == imp_row['unidad_concentracion']:
            contador += 1
        else:
            if imp_row['unidad_concentracion'] == 'suc':
                dismatch += 'suc' if dismatch == '' else '|suc'
            else:
                dismatch += 'uc' if dismatch == '' else '|uc'
        if medic['PRESENTACION'] == imp_row['presentacion']:
            contador += 1
        else:
            if imp_row['presentacion'] == 'sp':
                dismatch += 'sp' if dismatch == '' else '|sp'   
            else:
                dismatch += 'pp' if dismatch == '' else '|pp'
        if medic['FORMA FARMA'] == imp_row['forma_farma']:
            contador += 1
        else:
            if imp_row['forma_farma'] == 'sff':
                dismatch += 'sff' if dismatch == '' else '|sff'
            else:
                dismatch += 'ff' if dismatch == '' else '|ff'

        #si hay coincidencias y es mayor en numero de estas que el match anterior(en caso de que halla)
        if contador > coincidencias:
            coincidencias = contador
            matches = 1     
            medicine_match = str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch = dismatch
            match_ind = [i]
        #si existen las mismas coincidencias que el match anterior 
        elif contador > 0 and contador == coincidencias:
            matches += 1
            medicine_match += " || " + str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch += " || " + dismatch
            match_ind.append(i)

    c_r_c = f"{coincidencias} coincidencias en {matches} matches: {gb_dismatch}"

    if matches in (1,2,3) and coincidencias == 3:
        ind = match_ind[0]
        #se fuerza forma farma
        if gb_dismatch == 'ff':#se fuerza forma farma
            msj = f"se forzó su forma farma de {imp_row['forma_farma']} a {df_dict[ind]['FORMA FARMA']} dado el caso {c_r_c}"
            return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
                    imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
                    imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],
                    df_dict[ind]['EMPRESA'], df_dict[ind]['CONCENTRACION'], df_dict[ind]['UNIDAD CONCENTRACION'],
                    df_dict[ind]['PRESENTACION'], df_dict[ind]['FORMA FARMA'], df_dict[ind]['CNIS'], df_dict[ind]['STATUS'],'forzed perfect match',msj)

        #detección de nueva presentacion de un medicamento existente que varia en su concentracion o presentacion 
        elif gb_dismatch in ('pp', 'pp || pp', 'pp || pp || pp', 'cc', 'cc || cc', 'cc || cc || cc','pp || cc'):
            new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(imp_row['concentracion']) +' '+ 
                            imp_row['unidad_concentracion'] +' '+ str(imp_row['presentacion']) +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
            if new_medicine not in nuevos_medicamentos:   
                nuevos_medicamentos[new_medicine] = 1
            else:
                nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine] + 1
           
    
    elif matches in (1,2,3) and coincidencias == 2:
        #detección de nueva presentacion de un medicamento existente que varia en su concentracion o presentacion  y su forma_farma(esta se fuerza)
        if gb_dismatch in ('pp|ff || pp|ff', 'cc|ff', 'cc|ff || cc|ff', 'pp|ff', 'cc|pp || cc|pp || cc|pp'):
            ind = match_ind[0]
            new_medicine = (imp_row['nombre'] +' '+ imp_row['marca'] +' '+ df_dict[ind]['EMPRESA'] +' '+ str(imp_row['concentracion']) +' '+ 
                            imp_row['unidad_concentracion'] +' '+ str(imp_row['presentacion']) +' '+ df_dict[ind]['FORMA FARMA'] +' '+ str(df_dict[ind]['STATUS']))
            if new_medicine not in nuevos_medicamentos:
                nuevos_medicamentos[new_medicine] = 1
            else:
                nuevos_medicamentos[new_medicine] = nuevos_medicamentos[new_medicine]+1

    
    return (imp_row['unidades'],imp_row['tipo_unidad'],imp_row['valor_unitario'],
            imp_row['nombre'], imp_row['marca'], imp_row['descripcion_comercial'],imp_row['presentacion'],
            imp_row['forma_farma'], imp_row['concentracion'], imp_row['unidad_concentracion'],None,None,None,None,None,None,None,c_r_c,None)

In [24]:
avertl = imports[imports['nombre_y_marca']=='NM'].apply(match_imports_medicine_2, axis=1, result_type='expand').copy()

In [62]:
avertl[(avertl[17] != 'perfect match') & (avertl[17] != 'forzed perfect match')].to_csv('conflictos.csv')

In [ ]:
avertl[avertl[3] == 'pertuzumab']

In [34]:
avertl[17].value_counts()

perfect match                                                                                                                         567
forzed perfect match                                                                                                                  329
2 coincidencias en 1 matches: sp|sff                                                                                                  111
3 coincidencias en 1 matches: sp                                                                                                       60
1 coincidencias en 8 matches: sc|suc|sp || sc|suc|sp || sc|suc|sp || sc|suc|sp || sc|suc|sp || sc|suc|sp || sc|suc|sp || sc|suc|sp     55
2 coincidencias en 1 matches: cc|ff                                                                                                    33
0 coincidencias en 0 matches:                                                                                                          31
2 coincidencias en 3 matches: sp|f

In [26]:
nuevos_medicamentos

{'sunitinib sutent 28 Pfizer 50 mg 28 tabletas Generico': 5,
 'sunitinib sutent 28 Pfizer 25 mg 28 tabletas Generico': 6,
 'acalabrutinib calquence Astrazeneca 100 mg 30 tabletas Patente/Fuente Unica': 1,
 'acalabrutinib calquence Astrazeneca 100 mg 200 tabletas Patente/Fuente Unica': 1,
 'acalabrutinib calquence Astrazeneca 100 mg 60 tabletas Patente/Fuente Unica': 3,
 'alectinib alecensa F. Hoffmann-La Roche 150 mg 56 tabletas Patente/Fuente Unica': 3,
 'vandetanib caprelsa Sanofi Aventis 300 mg 30 comprimidos Patente/Fuente Unica': 3,
 'abemaciclib verzenio Eli Lilly 200 mg 28 tabletas Patente/Fuente Unica': 3,
 'dasatinib sprycel Bristol Myers Squibb 100 mg 30 tabletas Patente/Fuente Unica': 2,
 'larotrectinib vitrakvi Bayer 100 mg 100 capsulas nan': 1,
 'axitinib inlyta Pfizer 1 mg 60 tabletas Patente/Fuente Unica': 1,
 'pertuzumab perjeta F. Hoffmann-La Roche 30 mg 1 solucion inyectable Patente/Fuente Unica': 19,
 'polatuzumab polivy Genentech 140 mg 1 solucion inyectable Patente

## match perfecto (para probar)

In [22]:
def perfect_match_imports_medicine(imp_row: pd.Series) -> list:
    """ para generar la primera version de match perfectos para prueba"""
    #checamos si hace match perfecto(nombre,marca,presentacion etc)
    primer_intento =  medicines[(medicines['MEDICAMENTO']==imp_row['nombre']) & 
                                (medicines['NOMBRE COMERCIAL']==imp_row['marca']) &
                                (medicines['CONCENTRACION']==imp_row['concentracion']) & 
                                (medicines['UNIDAD CONCENTRACION']==imp_row['unidad_concentracion']) & 
                                (medicines['PRESENTACION']==imp_row['presentacion']) & 
                                (medicines['FORMA FARMA']==imp_row['forma_farma']) ].to_dict('records')
    if len(primer_intento) > 0:
        return (imp_row['forma_farma'],primer_intento[0]['EMPRESA'],  primer_intento[0]['CNIS'], primer_intento[0]['STATUS'],
                primer_intento[0]['UNIDOSIS'], 'perfect match', primer_intento[0]['INDICACION'], primer_intento[0]['INDICACIÓN CORTA'], None)
    
    #si no hizo match perfecto hay que ver que le falta y cuantos match con el mayor numero de coincidencias hace
    df_dict = medicines[ (medicines['MEDICAMENTO']==imp_row['nombre']) & (medicines['NOMBRE COMERCIAL']==imp_row['marca']) ].to_dict('records')
    if len(df_dict) == 0:
        #print(f"sin match para{imp_row['nombre']}, {imp_row['marca']}")
        return (imp_row['forma_farma'],None,None,None,None,'sin match',None,None,None)

    matches = 0 #número de matches entre imports y medicamentos con el mismo numero de coincidencias
    coincidencias = 0 #coincidencia entre imports y medicamentos en sus columnas, concentracion,unidad_concentracion, presentacion, forma farma
    medicine_match = '' #concatenacion del match o matches según sea el caso
    gb_dismatch = '' #es una concatenacion de pares claves que indican donde se fdió la diferencia en el match o matches
    match_ind = [] 
    global nuevos_medicamentos #genera un arreglo con medicamento y su número de apariciones
    for i,medic in enumerate(df_dict):
        contador = 0
        dismatch = ''
        if medic['CONCENTRACION'] == imp_row['concentracion']:
            contador += 1
        else:
            dismatch = 'cc' 
        if medic['UNIDAD CONCENTRACION'] == imp_row['unidad_concentracion']:
            contador += 1
        else:
            dismatch += 'uc' if dismatch == '' else '|uc'
        if medic['PRESENTACION'] == imp_row['presentacion']:
            contador += 1
        else:
            dismatch += 'pp' if dismatch == '' else '|pp'
        if medic['FORMA FARMA'] == imp_row['forma_farma']:
            contador += 1
        else:
            dismatch += 'ff' if dismatch == '' else '|ff'

        #si hay coincidencias y es mayor en numero de estas que el match anterior(en caso de que halla)
        if contador > coincidencias:
            coincidencias = contador
            matches = 1     
            medicine_match = str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch = dismatch
            match_ind = [i]
        #si existen las mismas coincidencias que el match anterior 
        elif contador > 0 and contador == coincidencias:
            matches += 1    
            medicine_match += " || " + str(medic['PRESENTACION']) +  medic['FORMA FARMA'] + str(medic['CONCENTRACION']) + medic['UNIDAD CONCENTRACION']
            gb_dismatch += " || " + dismatch
            match_ind.append(i)

    c_r_c = f"{coincidencias} coincidencias en {matches} matches: {medicine_match}"
    if matches == 1 and coincidencias == 3 and gb_dismatch == 'ff':
        ind = match_ind[0]
        msj = f"se forzó su forma farma de {imp_row['forma_farma']} a {df_dict[ind]['FORMA FARMA']} dado el caso {c_r_c}"
        return (df_dict[ind]['FORMA FARMA'], df_dict[ind]['EMPRESA'],  df_dict[ind]['CNIS'], df_dict[ind]['STATUS'],
                df_dict[ind]['UNIDOSIS'], 'forzed perfect match', df_dict[ind]['INDICACION'], df_dict[ind]['INDICACIÓN CORTA'], msj)


In [23]:
complete_imports = pd.read_csv('../cp_complete.csv')

In [24]:
complete_imports['concentracion'] =  complete_imports['concentracion'].astype(str).str.replace(".0","",regex=False)

In [25]:
complete_imports[['forma_farma','empresa','cnis','status_medicamento', 'unidosis','tipo_match','indicacion','indicacion_corta','c_r_c_llenado_datos']] = complete_imports[complete_imports['nombre_y_marca']=='NM'].apply(perfect_match_imports_medicine, axis=1, result_type='expand')

In [ ]:
complete_imports['tipo_match'].value_counts(dropna=False)

In [27]:
complete_imports[(complete_imports['tipo_match'] == 'perfect match') | (complete_imports['tipo_match'] == 'forzed perfect match') ].to_csv('perfect_match_v3.csv')

## prueba precios

In [15]:
pm = pd.read_csv('perfect_match_v2.csv')

In [45]:
avertl

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
9,102.0,PZA,2093.409,olaparib,lynparza,lynparza 50 mg capsulas (olaparib),sp,capsulas,50,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 112capsulas50mg,None
10,406.0,PZA,204.067,gefitinib,iressa,iressa 250 mg tabletas (gefitinib),sp,tabletas,250,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 30tabletas250mg,None
17,40.0,PZA,367.477,erlotinib,tarceva,tarceva 100 mg comprimidos (erlotinib),sp,comprimidos,100,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 30comprimidos100mg,None
20,608.0,PZA,296.529,lapatinib,tykerb,tykerb 250 mg tabletas (lapatinib),sp,tabletas,250,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 70tabletas250mg,None
22,1475.0,PZA,1307.024,nilotinib,tasigna,tasigna 200 mg capsulas (nilotinib),sp,capsulas,200,mg,None,None,None,None,None,None,None,2 coincidencias en 3 matches: 112comprimidos20...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,270.0,PZA,1645.774,olaparib,lynparza,lynparza 50 mg capsulas (olaparib),sp,capsulas,50,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 112capsulas50mg,None
710,54.0,PZA,317.027,erlotinib,tarceva,tarceva 100 mg comprimidos (erlotinib),sp,comprimidos,100,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 30comprimidos100mg,None
713,340.0,PZA,265.749,lapatinib,tykerb,tykerb 250 mg tabletas (lapatinib),sp,tabletas,250,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 70tabletas250mg,None
737,247.0,PZA,236.304,gefitinib,iressa,iressa 250 mg tabletas (gefitinib),sp,tabletas,250,mg,None,None,None,None,None,None,None,3 coincidencias en 1 matches: 30tabletas250mg,None


In [51]:
pm[ (pm['nombre']=='lapatinib') & 
    (pm['marca']=='tykerb') &
    (pm['concentracion']==250) & 
    (pm['unidad_concentracion']=='mg') & 
    (pm['forma_farma']=='tabletas') ]

,Unnamed: 0.1,Unnamed: 0,Partida Aduanera,Fecha,Importador,Exportador,unidades,tipo_unidad,valor_total,valor_unitario,...,status_medicamento,presentacion,forma_farma,nombre_medicamento,empresa,unidosis,tipoI,tipoII,c_r_c_llenado_datos,tipo_match
333,485,1047,30049099,44238,(NFA971101EDA) NOVARTIS FARMACEUTICA SA DE CV,"FISHER CLINICAL SERVICES, INC",3.0,PZA,35.41,11.803,...,Patente/Fuente Unica,70,tabletas,NaN,Glaxosmithkline,17500.0,NaN,NaN,NaN,perfect match


In [78]:
def prueba_precios(imp_row):
    perfect_matches = pm[(pm['nombre']==imp_row[3]) & 
                        (pm['marca']==imp_row[4]) &
                        (pm['concentracion']==imp_row[8]) & 
                        (pm['unidad_concentracion']==imp_row[9]) & 
                        (pm['forma_farma']==imp_row[7]) ].to_dict('records')
    valores_unitarios = ''
    for perfect_match in perfect_matches:
        valores_unitarios += perfect_match['tipo_unidad']+str(perfect_match['valor_unitario'])

    return  valores_unitarios

In [79]:
avertl['valores_unitarios'] = avertl.apply(prueba_precios,axis=1)

In [81]:
avertl.to_csv('prueba.csv')

In [73]:
pm['concentracion'] =  pm['concentracion'].astype(str).str.replace(".0","",regex=False)